In [ ]:
import pandas as pd
from tqdm import tqdm

file_names = []
outputs = []
output_paths = [
    '/opt/ml/input/code/submission/voting/MVT.csv',
    '/opt/ml/input/code/submission/voting/Segformerb4.csv',
    '/opt/ml/input/code/submission/voting/Segformerb5.csv'
]

for output_path in output_paths:
    data = pd.read_csv(output_path)
    outputs.append(list(data['PredictionString']))
    file_names.append(list(data['image_id']))

In [16]:
import numpy as np

l_t = np.array([0,1,2,3,4,5,6,7,8])
l_p = np.array([0,1,5,7,8,9,1,0,6])

mask = (l_t >= 0) & (l_t < 9)

# hist = np.bincount(
# 				9 * l_t[mask].astype(int) + l_p[mask],
#         minlength=9 ** 2
# 		).reshape(9, 9)


(array([ 0,  9, 18, 27, 36, 45, 54, 63, 72]),
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True]),
 array([0, 1, 5, 7, 8, 9, 1, 0, 6]))

In [17]:
results = []
weights = [0.4, 0.3, 0.3]

for o1, o2, o3 in tqdm(zip(outputs[0], outputs[1], outputs[2])):
    result = []
    for i, j, k in zip(o1.split(' '), o2.split(' '), o3.split(' ')):
        i, j, k = int(i), int(j), int(k)
        
        if int(i + j + k) > 33:
            print(i, j, k, str(round((int(i+j+k))/len(outputs))))
            
        result.append(str(round((int(i+j+k))/len(outputs))))
        
    results.append(result)

819it [01:31,  8.95it/s]


In [18]:
import os

print("Start Inference")
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)

# PredictionString 대입
for file_name, array in zip(file_names[0], results):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(array)}, 
                                ignore_index=True)
    
submission.to_csv(os.path.join('/opt/ml/input/code/submission/voting', 'submission.csv'), index=False)
print("End Voting.")

Start Inference
End Voting.


In [2]:
import numpy as np
import pydensecrf.densecrf as dcrf
import matplotlib.pyplot as plt
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral, create_pairwise_gaussian
import cv2
from skimage.color import gray2rgb
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
%matplotlib inline

/opt/conda/envs/torch/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
X=cv2.imread("C:/Aashish/Assignments/Machine Learning/Aashish/input/rampur_irs2l4_2may2017.tif")
y=cv2.imread("C:/Aashish/Assignments/Machine Learning/Aashish/label/rampur_burned.tif")
# out=cv2.imread("C:\Aashish\Assignments\Machine Learning\CRF - DataSet\20_actual.png")

In [ ]:
"""
Function which returns the labelled image after applying CRF

"""

#Original_image = Image which has to labelled
#Annotated image = Which has been labelled by some technique( FCN in this case)
#Output_image = Name of the final output image after applying CRF
#Use_2d = boolean variable 
#if use_2d = True specialised 2D fucntions will be applied
#else Generic functions will be applied

def crf(original_image, annotated_image, output_image, use_2d = True):
    
    # Converting annotated image to RGB if it is Gray scale
    if(len(annotated_image.shape)<3):
        annotated_image = gray2rgb(annotated_image).astype(np.uint32)
    
    cv2.imwrite("testing2.png",annotated_image)
    annotated_image = annotated_image.astype(np.uint32)
    #Converting the annotations RGB color to single 32 bit integer
    annotated_label = annotated_image[:,:,0].astype(np.uint32) + (annotated_image[:,:,1]<<8).astype(np.uint32) + (annotated_image[:,:,2]<<16).astype(np.uint32)
    
    # Convert the 32bit integer color to 0,1, 2, ... labels.
    colors, labels = np.unique(annotated_label, return_inverse=True)
    
    #Creating a mapping back to 32 bit colors
    colorize = np.empty((len(colors), 3), np.uint8)
    colorize[:,0] = (colors & 0x0000FF)
    colorize[:,1] = (colors & 0x00FF00) >> 8
    colorize[:,2] = (colors & 0xFF0000) >> 16
    
    #Gives no of class labels in the annotated image
    n_labels = len(set(labels.flat)) 
    
    print("No of labels in the Image are ")
    print(n_labels)
    
    
    #Setting up the CRF model
    if use_2d :
        d = dcrf.DenseCRF2D(original_image.shape[1], original_image.shape[0], n_labels)

        # get unary potentials (neg log probability)
        U = unary_from_labels(labels, n_labels, gt_prob=0.90, zero_unsure=False)
        d.setUnaryEnergy(U)

        # This adds the color-independent term, features are the locations only.
        d.addPairwiseGaussian(sxy=(3, 3), compat=3, kernel=dcrf.DIAG_KERNEL,
                          normalization=dcrf.NORMALIZE_SYMMETRIC)

        # This adds the color-dependent term, i.e. features are (x,y,r,g,b).
        d.addPairwiseBilateral(sxy=(80, 80), srgb=(13, 13, 13), rgbim=original_image,
                           compat=10,
                           kernel=dcrf.DIAG_KERNEL,
                           normalization=dcrf.NORMALIZE_SYMMETRIC)
        
    #Run Inference for 5 steps 
    Q = d.inference(5)

    # Find out the most probable class for each pixel.
    MAP = np.argmax(Q, axis=0)

    # Convert the MAP (labels) back to the corresponding colors and save the image.
    # Note that there is no "unknown" here anymore, no matter what we had at first.
    MAP = colorize[MAP,:]
    cv2.imwrite(output_image,MAP.reshape(original_image.shape))
    return MAP.reshape(original_image.shape)

In [ ]:
crfimage = crf(X,y,"out.png")

In [ ]:
plt.subplot(1,2,1)
plt.imshow(y)
plt.subplot(1,2,2)
plt.imshow(crfimage)

In [4]:
import glob
from PIL import Image
import matplotlib as plt
import numpy as np

path = '/opt/ml/input/data/batch_03_vt/*'
image_paths = glob.glob(path)

for i, image_path in enumerate(image_paths):
    image = np.array(Image.open(image_path))
    print(image.shape)
    
    if i == 20:
        break